# Plot visulization

In [4]:
# ────────────────────────────────────────────────────────────────────────────────
# 0. Imports & reproducibility
# ────────────────────────────────────────────────────────────────────────────────
import math, random, numpy as np, torch
from torch import nn
# Basic
import torch
import math
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
from numpy import random
torch.set_num_threads(4)
import numpy as np
import matplotlib.pyplot as plt



# PDEi
from utils_pde.utils_pde_2dpoisson import Poisson2D

# Viz
from utils_tools.utils_result_viz import plot_predictions_2D
from utils_tools.utils_tuning import save_plot

# Base Mdoels
from utils_uqmd.utils_uq_dropout import DropoutPINN
from utils_uqmd.utils_uq_mlp import MLPPINN
from utils_uqmd.utils_uq_vi import VIBPINN
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# CP
from utils_uqmd.utils_uq_cp import CP

torch.set_num_threads(4)
seed = 5902
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# ────────────────────────────────────────────────────────────────────────────────
# 1. Ground-truth PDE setup 
# ────────────────────────────────────────────────────────────────────────────────
from utils_pde.utils_pde_allencahn2d import AllenCahn2D

# 1) keep the torch-based definition
def true_solution(xy):
    return torch.sin(math.pi*xy[...,0:1]) * torch.sin(math.pi*xy[...,1:2])

domain=((-1.0, 1.0), (-1.0, 1.0))
pde = AllenCahn2D(lam=0.05, domain=domain, true_solution=true_solution)

# ────────────────────────────────────────────────────────────────────────────────
# 2. Data Generation (Noisy data for training, test, calibration)
# ────────────────────────────────────────────────────────────────────────────────
colloc_pt_num = 250  # Number of collocation points

data_noise = 0.3
N_train = 500
N_test = 200
N_calib = 100
N_valid = 100

N_total= 50
N_colloc = 100
N_boundary= 20

X_train, Y_train = pde.data_generation(N_train, data_noise)
X_test, Y_test = pde.data_generation(N_test, data_noise)
X_calibration, Y_calibration = pde.data_generation(N_calib, data_noise)
X_validation, Y_validation= pde.data_generation(N_valid, data_noise)

X_vis, Y_vis = pde.data_generation(N_total, data_noise)
X_coloc, Y_coloc = pde.data_generation(N_colloc, data_noise)


def make_dense_grid(domain, N=200):
    x = np.linspace(domain[0][0], domain[0][1], N)
    y = np.linspace(domain[1][0], domain[1][1], N)
    X, Y = np.meshgrid(x, y)
    XY_grid = np.stack([X.ravel(), Y.ravel()], axis=-1)
    return XY_grid, X, Y

# 4) plot again
grid_test, X_mesh, Y_mesh = make_dense_grid(domain, 200)
grid_test_torch = torch.tensor(grid_test, dtype=torch.float32)
U_true_grid = true_solution(grid_test_torch).numpy().reshape(X_mesh.shape)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

def plot_truth_and_samples_2D(
    X_train, Y_train, grid, U_true_grid, domain,
    title="2D PDE samples visualization",
    dot_size=46,            # size of the dots
    edge_lw=2.5,            # black edge thickness
    use_halo=False           # white halo under the black edge
):
    fig, ax = plt.subplots(figsize=(7, 6))

    # Background field (viridis), no contour lines
    im = ax.imshow(
        U_true_grid,
        extent=(domain[0][0], domain[0][1], domain[1][0], domain[1][1]),
        origin='lower',
        aspect='auto',
        cmap='viridis',
        alpha=1.0
    )
    cmap = im.cmap
    norm = im.norm  # share exact scaling with the background

    # Optional halo to boost edge visibility on any background
    if use_halo:
        ax.scatter(
            X_train[:, 0], X_train[:, 1],
            facecolors='none',
            edgecolors='white',
            linewidths=edge_lw + 1.4,  # thicker under-stroke
            marker='o',
            s=dot_size,
            zorder=3
        )

    # Value-colored dots with bold, dark edges
    ax.scatter(
        X_train[:, 0], X_train[:, 1],
        c=Y_train,
        cmap=cmap,
        norm=norm,
        marker='o',
        s=dot_size,
        edgecolors='black',
        linewidths=edge_lw,
        alpha=1.0,
        zorder=3.1,
        label="Noisy samples"
    )

    # One shared colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label("u(x, y)")

    ax.set_xlabel("x"); ax.set_ylabel("y")
    ax.set_title(title)
    plt.tight_layout()
    return fig, ax


save_plot(
    plot_truth_and_samples_2D,
    save_dir="Plot_procedure", prefix="2D_AllenCahn_data",
)(
    X_vis, Y_vis, grid_test, U_true_grid, domain,
    title="Allen-Cahn 2D: ground-truth vs noisy data"
)

import numpy as np
import matplotlib.pyplot as plt

def plot_truth_and_coloc_2D(
    X_train, Y_train, grid, U_true_grid, domain,
    title="2D PDE samples visualization",
    N_c=200,                 # total desired collocation points
    include_boundary=False,   # <- default: NO boundary points
    marker='x', color='red',  # style for collocation points
    s=40, lw=2.5
):
    """
    Show the true field (viridis) and overlay N_c uniformly placed collocation points.
    (X_train, Y_train) are ignored; kept for wrapper compatibility.
    Returns (fig, ax, X_c) where X_c.shape == (N_actual, 2).
    """
    (x0, x1), (y0, y1) = domain
    w, h = float(x1 - x0), float(y1 - y0)
    assert w > 0 and h > 0, "domain must have positive width and height"

    # Choose grid counts to approximate N_c while respecting aspect ratio
    N_c = int(max(1, N_c))
    nx = max(2, int(np.round(np.sqrt(N_c * (w / h)))))
    ny = max(2, int(np.ceil(N_c / nx)))
    N_actual = nx * ny

    # Build coordinates
    if include_boundary:
        xs = np.linspace(x0, x1, nx)
        ys = np.linspace(y0, y1, ny)
    else:
        # Strictly interior: drop both endpoints (no points on the boundary)
        xs = np.linspace(x0, x1, nx + 2)[1:-1]
        ys = np.linspace(y0, y1, ny + 2)[1:-1]

    Xg, Yg = np.meshgrid(xs, ys, indexing='xy')
    X_c = np.column_stack([Xg.ravel(), Yg.ravel()])

    # Plot field
    fig, ax = plt.subplots(figsize=(7, 6))
    im = ax.imshow(
        U_true_grid,
        extent=(x0, x1, y0, y1),
        origin='lower',
        aspect='auto',
        cmap='viridis',
        alpha=1.0,
        zorder=0
    )

    # Plot collocation points (interior only by default)
    ax.scatter(
        X_c[:, 0], X_c[:, 1],
        marker=marker, c=color, s=s, linewidths=lw,
        label=f"Collocation (N={N_actual}, interior={not include_boundary})",
        zorder=2
    )

    # Decor
    plt.colorbar(im, ax=ax, label="u(x, y)")
    ax.set_xlim(x0, x1); ax.set_ylim(y0, y1)
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel("x"); ax.set_ylabel("y")
    ax.set_title(title)
    plt.tight_layout()
    return fig, ax, X_c



save_plot(
    plot_truth_and_coloc_2D,
    save_dir="Plot_procedure", prefix="2D_AllenCahn_colloc_data",
)(
    X_coloc, Y_coloc, grid_test, U_true_grid, domain,
    title="Allen-Cahn 2D: ground-truth & collocation points"
)

import numpy as np
import matplotlib.pyplot as plt

def plot_truth_with_boundary_2D(
    X_dummy, Y_dummy, grid_dummy, U_true_grid, domain,
    title="2D PDE: field + boundary points",
    N_b=200,
    dot_size=200,
    edge_lw=4,
    use_halo=False,
    equal_per_side=False,
):
    """
    Show the true field (viridis) and overlay N_b boundary points along `domain`.
    Returns (fig, ax, X_b) where X_b.shape == (N_b, 2).
    """

    # --- parse domain ---
    try:
        (x0, x1), (y0, y1) = domain
    except Exception as e:
        raise ValueError(
            f"`domain` must be ((x0, x1), (y0, y1)); got {type(domain)} with value {domain}"
        ) from e

    w, h = (x1 - x0), (y1 - y0)
    assert w > 0 and h > 0, "domain must have positive width and height"
    N_b = int(max(1, N_b))

    # --- boundary generators ---
    def _points_perimeter(N):
        """Uniform by arc length, avoiding exact corners."""
        L = 2.0 * (w + h)
        t = np.linspace(0.0, L, N, endpoint=False) + (L / (2.0 * N))
        xs = np.empty_like(t); ys = np.empty_like(t)
        for i, ti in enumerate(t):
            if ti < w:                       # bottom: left -> right
                xs[i] = x0 + ti;         ys[i] = y0
            elif ti < w + h:                 # right: bottom -> top
                xs[i] = x1;               ys[i] = y0 + (ti - w)
            elif ti < 2*w + h:               # top: right -> left
                xs[i] = x1 - (ti - (w + h)); ys[i] = y1
            else:                            # left: top -> bottom
                xs[i] = x0;               ys[i] = y1 - (ti - (2*w + h))
        return np.column_stack([xs, ys])

    def _points_equal_sides(N):
        """Roughly equal count per side; corners once."""
        n_side = max(1, N // 4)
        extras = N - 4 * n_side
        counts = [n_side] * 4
        for i in range(extras):
            counts[i] += 1

        xb = np.linspace(x0, x1, counts[0], endpoint=False); yb = np.full_like(xb, y0)
        yr = np.linspace(y0, y1, counts[1], endpoint=False); xr = np.full_like(yr, x1)
        xt = np.linspace(x0, x1, counts[2], endpoint=False) + (w / max(counts[2], 1)) * 0.5
        yt = np.full_like(xt, y1)
        yl = np.linspace(y0, y1, counts[3], endpoint=False) + (h / max(counts[3], 1)) * 0.5
        xl = np.full_like(yl, x0)

        Xp = np.concatenate([
            np.column_stack([xb, yb]),
            np.column_stack([xr, yr]),
            np.column_stack([xt, yt]),
            np.column_stack([xl, yl]),
        ], axis=0)
        if Xp.shape[0] != N:
            Xp = _points_perimeter(N)
        return Xp

    X_b = _points_equal_sides(N_b) if equal_per_side else _points_perimeter(N_b)

    # --- plot field + boundary points ---
    fig, ax = plt.subplots(figsize=(7, 6))

    im = ax.imshow(
        U_true_grid,
        extent=(x0, x1, y0, y1),
        origin="lower",
        aspect="auto",
        cmap="viridis",
        alpha=1.0,
        zorder=0
    )

    # high-visibility ring markers (no fill), with optional white halo
    if use_halo:
        ax.scatter(
            X_b[:, 0], X_b[:, 1],
            facecolors="none", edgecolors="white",
            linewidths=edge_lw + 1.4, marker="o", s=dot_size, zorder=2
        )
    ax.scatter(
        X_b[:, 0], X_b[:, 1],
        facecolors="none", edgecolors="red",
        linewidths=edge_lw, marker="o", s=dot_size, zorder=3,
        label=f"Boundary (N_b={N_b})"
    )

    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label("u(x, y)")

    ax.set_xlim(x0, x1); ax.set_ylim(y0, y1)
    ax.set_aspect("equal", adjustable="box")
    ax.set_xlabel("x"); ax.set_ylabel("y")
    ax.set_title(title)
    plt.tight_layout()
    return fig, ax, X_b


save_plot(
    plot_truth_with_boundary_2D,
    save_dir="Plot_procedure", prefix="2D_AllenCahn_boudnary_points",
)(
    X_coloc, Y_coloc, grid_test, U_true_grid, domain,
    title = "Allen-Cahn 2D: ground true & boundary points", 
    N_b=100
)



<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>